In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn import preprocessing

params = {}

sns.set_theme(style="whitegrid", palette="colorblind", rc=params)

/tmp/ipykernel_1228746/2527920795.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_stata("../data/mothers_fertility_timing.dta")

In [3]:
df.describe()

,iir01,irh04_a,irh04_b,irh04_c,irh06_a,irh06_b,irh06_c,irh12_a,irh12_b,irh12_c,...,lack_health_fac_top3_bl,lack_med_equip_top3_bl,lack_hlthworkers_top3_bl,low_hlth_aware_top3_bl,lack_sch_fac_top3_bl,lack_sch_infr_top3_bl,lack_sch_teachers_top3_bl,lack_sch_aware_top3_bl,births_since_baseline_all_v2,births_since_baseline_nosbmc_v2
count,46428.000000,29382.000000,29382.000000,43588.000000,9564.000000,9564.000000,12397.000000,5242.000000,5242.000000,7559.000000,...,46369.000000,46369.000000,46369.000000,46369.000000,46369.000000,46369.000000,46369.000000,46369.000000,31670.000000,31670.000000
mean,35.600327,1.219522,1.090940,2.360374,0.739753,0.887286,1.598209,0.780809,0.613888,1.400053,...,0.240700,0.166038,0.316332,0.096185,0.234877,0.383769,0.221419,0.107141,0.205968,0.190875
std,8.829804,1.044989,0.964942,1.358538,0.812254,0.820021,0.946183,0.742982,0.704404,0.830282,...,0.427513,0.372118,0.465049,0.294848,0.423926,0.486308,0.415207,0.309295,0.473007,0.450427
min,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,36.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,42.000000,2.000000,2.000000,3.000000,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,76.000000,8.000000,9.000000,13.000000,7.000000,6.000000,9.000000,6.000000,6.000000,12.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,4.000000,4.000000


In [5]:
outcomes = ["births_since_baseline_all_v2", "births_since_baseline_nosbmc_v2", ]

In [6]:
assigment_treatment = ["L07"]

In [16]:
baseline_covs = []
for col in df.columns:
    if "baseline_nm" in col:
        baseline_covs.append(col)
    elif "_miss" in col:
        baseline_covs.append(col)

# Todo Kabupaten

In [22]:
df_x = df[baseline_covs]
for col in df_x.columns:
    df_x.loc[:, col] = df_x[col].fillna(df_x[col].median())

In [23]:
df_x.describe()

,hh_educ_1_baseline_miss,hh_educ_1_baseline_nm,hh_educ_2_baseline_miss,hh_educ_2_baseline_nm,hh_educ_3_baseline_miss,hh_educ_3_baseline_nm,hh_educ_4_baseline_miss,hh_educ_4_baseline_nm,hh_educ_5_baseline_miss,hh_educ_5_baseline_nm,...,electricity_PLN_baseline_miss,electricity_PLN_baseline_nm,logpcexp_baseline_miss,logpcexp_baseline_nm,logpcexp_food_baseline_miss,logpcexp_food_baseline_nm,logpcexp_nonfood_baseline_miss,logpcexp_nonfood_baseline_nm,hhsize_ln_baseline_miss,hhsize_ln_baseline_nm
count,46443.000000,46443.0,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,...,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000,46443.000000
mean,0.023233,0.0,0.023233,0.200504,0.023233,0.482204,0.023233,0.113386,0.023233,0.055251,...,0.023233,0.836811,0.023233,11.738033,0.023534,11.386110,0.023233,10.412446,0.023233,1.589393
std,0.150644,0.0,0.150644,0.400382,0.150644,0.499689,0.150644,0.317068,0.150644,0.228471,...,0.150644,0.369542,0.150644,1.871173,0.151594,1.829361,0.150644,1.758631,0.150644,0.415565
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,11.694388,0.000000,11.342814,0.000000,10.184334,0.000000,1.386294
50%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,12.001481,0.000000,11.654239,0.000000,10.631942,0.000000,1.609438
75%,0.000000,0.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,12.304696,0.000000,11.952562,0.000000,11.080263,0.000000,1.791759
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,14.363928,1.000000,13.663857,1.000000,14.258967,1.000000,3.135494


In [24]:
df_y = df[outcomes[0]]

In [25]:
df_y.describe()

count    31670.000000
mean         0.205968
std          0.473007
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          4.000000
Name: births_since_baseline_all_v2, dtype: float64

In [39]:
df_t = pd.get_dummies(df[assigment_treatment], drop_first=True)

In [28]:
idx = df_y.notna()

In [46]:
x = df_x[idx].to_numpy().astype("float32")
y = df_y[idx].to_numpy().astype("float32")
t = df_t[idx].to_numpy().astype("float32")

In [47]:
print(x.shape, y.shape, t.shape)

(31670, 106) (31670,) (31670, 1)


In [48]:
from ml_field_experiments.effect_heterogeneity.methods.grf import GRFTest

In [49]:
result = GRFTest(w_hat=t.mean()).run(x=x, z=t, y=y)

In [50]:
print(result)

{'ate': 0.00023524725789352365, 'p-val-ate': 0.9631811389010745, 'p-val-het': 0.0, 'mse': nan, 'CATE': array([-0.00054018,  0.00411522,  0.00238836, ...,  0.01426394,
       -0.00130666,  0.00826689])}


In [51]:
print(result["CATE"].var())

0.0002181042852031196
